<center><table>
    <tr>
        <th><img src="./INL1.png",align="middle",height="10000"/></th>
        <th><img src="./MIT1.png",align="middle",height="10"\></th>
        <th><img src="./ANL.png",align="middle",height="10"/></th>
    </tr>
</table>
</center>

# <center>Cost Reduction Framework for the Sodium Fast Nuclear Reactor (SFR) Power Plant</center>
# <center>Botros Hanna</center>

## Section 0 : Essentials to Run the code

### Section 0 - 1 : Importing the libraries

In [1]:
# Importing libararies
import pandas as pd
import numpy as np
from tabulate import tabulate

#pd.set_option('mode.chained_assignment', None)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', None)

# Things to do
# does the number of orders have effect ? 
# bulk order reduction effec on all accounts?

# Include startup costs and duration
# Update relationships with Botros results
# Update indirect costs
# Update interest
# Update ITC
# Adjust impact of supplychain delays on construction duration per Botros document
# Take a look at why costs are so large with Botros equations
# How to reduce variables and levers from FOAK to NOAK? 

# My comments
# standardizaition applied?
# is it always 90% ? Even in the first reactor?
# do I need to redo the equations since I chantged the structure a little bit?
# maybe gradually decrease the other variables

### Section 0 - 2 : Control the table style

In [2]:
def prettify(database1, caption, category):
    database1['Total Cost (USD)'] = database1['Total Cost (USD)'].apply(lambda x: '$ {:,.0f}'.format(x))
    database1['Factory Equipment Cost'] = database1['Factory Equipment Cost'].apply(lambda x: '$ {:,.0f}'.format(x))
    database1['Site Labor Cost'] = database1['Site Labor Cost'].apply(lambda x: '$ {:,.0f}'.format(x))
    database1['Site Material Cost'] = database1['Site Material Cost'].apply(lambda x: '$ {:,.0f}'.format(x))
    database1['Site Labor Hours'] =database1['Site Labor Hours'].apply(lambda x: '{:,.0f} hrs'.format(x))
    
    database1 = database1.fillna('') # remove nan
    database1= database1.replace({'$ nan':'', 'nan hrs':''}) # remove nan


    df_ = database1.loc[database1.Account.isin([10, 20, 30, 40, 50, 60])] # highlight high level accounts

    df_2 = database1.iloc[-10:] # final results  # df_2 = database1.iloc[-10:]
    
    df3 = pd.concat([df_, df_2])
    df4 = df_2 = database1.iloc[-2:]
    slice_ = pd.IndexSlice[df3.index, df3.columns]
    slice_2 = pd.IndexSlice[df4.index, df4.columns]


    
    slice_ = pd.IndexSlice[df3.index, df3.columns]
    slice_2 = pd.IndexSlice[df4.index, df4.columns]
    
    if category == "no_subsidies":
        database_styled = (database1.style.set_properties(**{'font-weight': 'bold'}, subset=slice_).\
                           set_properties(**{'color': 'white','background-color': 'white' }, subset=slice_2).\
                           set_caption(caption).set_table_styles([{'selector': 'caption','props': [('color', 'red'),('font-size', '20px')]}]))
            
    elif category == "subsidies":
        database_styled = (database1.style.set_properties(**{'font-weight': 'bold'}, subset=slice_).set_caption(caption).\
                           set_table_styles([{'selector': 'caption','props': [('color', 'red'),('font-size', '20px')]}]))
    
    return database_styled.hide()

def highlight_changes(row, baseline):
    return ['background-color: yellow' if val != baseline\
            and len(str(val))>0 and not (str(val)).startswith("Range")  else '' for val in row]

## Section 1 : Reading the Baseline SFR Cost Summary Table 

In [3]:
# Reading excel or csv files
SFR_data_0 = pd.read_excel('SFR_allData_v2.xlsx', sheet_name = "SA_Plus_MIT_Combined", nrows= 72)

db = pd.DataFrame()
db = SFR_data_0[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()
SFR_data = db
SFR_data_pretty = prettify(SFR_data, "SFR-FOAK Capital Cost Summary - Baseline hypothetical well-executeed project (duration = 64 months) ", "no_subsidies")
SFR_data_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 11,000,000",,,,
12,Site Permits,"$ 1,598,891",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 78,991,565",,,,


## Section 2 : A function to update the cost summary table each time a change is made

In [4]:
# Update the high level costs in the database when changing the low level costs
reactor_power = 310.8 * 1000 # kw
def update_high_level_costs(db):


    # update account 21 : material, labor, factory
    db.loc[db.Account == 21, 'Factory Equipment Cost'] = (db.loc[db.Account == 212, 'Factory Equipment Cost']).values+\
    (db.loc[db.Account == 213, 'Factory Equipment Cost']).values +(db.loc[db.Account == '211 plus 214 to 219', 'Factory Equipment Cost']).values
   
    db.loc[db.Account == 21, 'Site Material Cost'] = (db.loc[db.Account == 212, 'Site Material Cost'].values)+\
    (db.loc[db.Account == 213, 'Site Material Cost']).values+(db.loc[db.Account == '211 plus 214 to 219', 'Site Material Cost']).values
    
    db.loc[db.Account == 21, 'Site Labor Cost'] = (db.loc[db.Account == 212, 'Site Labor Cost']).values+\
    (db.loc[db.Account == 213, 'Site Labor Cost']).values+(db.loc[db.Account == '211 plus 214 to 219', 'Site Labor Cost']).values
    
    db.loc[db.Account == 21, 'Site Labor Hours'] = (db.loc[db.Account == 212, 'Site Labor Hours']).values+\
    (db.loc[db.Account == 213, 'Site Labor Hours']).values+(db.loc[db.Account =='211 plus 214 to 219', 'Site Labor Hours']).values


    #     # update account 23 : material, labor, factory
    db.loc[db.Account == 23, 'Factory Equipment Cost'] = (db.loc[db.Account == '232.1', 'Factory Equipment Cost']).values+\
    (db.loc[db.Account == 233, 'Factory Equipment Cost']).values
    
    (db.loc[db['Account'] == 23, 'Site Material Cost']) = (db.loc[db['Account'] == '232.1', 'Site Material Cost']).values+\
    (db.loc[db['Account'] == 233, 'Site Material Cost']).values
    
    (db.loc[db['Account'] == 23, 'Site Labor Cost']) = (db.loc[db['Account'] == '232.1', 'Site Labor Cost']).values+\
    (db.loc[db['Account'] == 233, 'Site Labor Cost']).values
    
    (db.loc[db['Account'] == 23, 'Site Labor Hours']) = (db.loc[db['Account'] == '232.1', 'Site Labor Hours']).values+\
    (db.loc[db['Account'] == 233, 'Site Labor Hours']).values




    
    # update total costs for accounts 21 : 26
    
    # total = labor + factory + material
    for x in [21, 212, 213, '211 plus 214 to 219', 22, 23, '232.1', 233, 24, 26]: 
        (db.loc[db['Account'] == x, 'Total Cost (USD)']) = (db.loc[db['Account'] == x, 'Factory Equipment Cost'])+\
            (db.loc[db['Account'] == x, 'Site Labor Cost'])+ (db.loc[db['Account'] == x, 'Site Material Cost'])

    #update total costs for accounts 10
    (db.loc[db['Title'] == '10s - Subtotal', 'Total Cost (USD)']) =\
        db.loc[db['Account'].isin([11, 12, 13, 14, 15, 16, 18]), 'Total Cost (USD)'].sum()
     
    # update total costs for accounts 20
    (db.loc[db['Title'] == '20s - Subtotal', 'Total Cost (USD)']) =\
        db.loc[db['Account'].isin([21, 22, 23, 24, 25, 26, 28]), 'Total Cost (USD)'].sum()

    # update total costs for accounts 30
    (db.loc[db['Title'] == '30s - Subtotal', 'Total Cost (USD)']) =\
        db.loc[db['Account'].isin([31, 32, 33, 34, 35]), 'Total Cost (USD)'].sum()

    # update total costs for accounts 40
    (db.loc[db['Title'] == '40s - Subtotal', 'Total Cost (USD)']) =\
        db.loc[db['Account'].isin([41, 42]), 'Total Cost (USD)'].sum()

    # update total costs for accounts 50
    (db.loc[db['Title'] == '50s - Subtotal', 'Total Cost (USD)']) =\
        db.loc[db['Account'].isin([51, 52, 54]), 'Total Cost (USD)'].sum()

    # update total costs for accounts 60
    (db.loc[db['Title'] == '60s - Subtotal', 'Total Cost (USD)']) =\
        db.loc[db['Account'].isin([ 62]), 'Total Cost (USD)'].sum()
    
    # uodate costs per kw
    
    
    (db.loc[db['Title'] == '10s - $/kWe', 'Total Cost (USD)']) = (db.loc[db['Title'] == '10s - Subtotal', 'Total Cost (USD)']).values/reactor_power 
    (db.loc[db['Title'] == '20s - $/kWe', 'Total Cost (USD)']) = (db.loc[db['Title'] == '20s - Subtotal', 'Total Cost (USD)']).values/reactor_power 
    (db.loc[db['Title'] == '30s - $/kWe', 'Total Cost (USD)']) = (db.loc[db['Title'] == '30s - Subtotal', 'Total Cost (USD)']).values/reactor_power 
    (db.loc[db['Title'] == '40s - $/kWe', 'Total Cost (USD)']) = (db.loc[db['Title'] == '40s - Subtotal', 'Total Cost (USD)']).values/reactor_power 
    (db.loc[db['Title'] == '50s - $/kWe', 'Total Cost (USD)']) = (db.loc[db['Title'] == '50s - Subtotal', 'Total Cost (USD)']).values/reactor_power 
    (db.loc[db['Title'] == '60s - $/kWe', 'Total Cost (USD)']) = (db.loc[db['Title'] == '60s - Subtotal', 'Total Cost (USD)']).values/reactor_power 
    
    
    
    # update final results
    (db.loc[db['Title'] == 'Total Direct Capital Cost (Accounts 10 to 20)', 'Total Cost (USD)']) =\
        (db.loc[db['Title'] == '10s - Subtotal', 'Total Cost (USD)']).values + (db.loc[db['Title'] == '20s - Subtotal', 'Total Cost (USD)']).values
    
    (db.loc[db['Title'] == 'Base Construction Cost (Accounts 10 to 30)', 'Total Cost (USD)']) =\
       (db.loc[db['Title'] == 'Total Direct Capital Cost (Accounts 10 to 20)', 'Total Cost (USD)']).values +\
        (db.loc[db['Title'] == '30s - Subtotal', 'Total Cost (USD)']).values

    (db.loc[db['Title'] == 'Total Overnight Cost (Accounts 10 to 50)', 'Total Cost (USD)']) =\
        (db.loc[db['Title'] == 'Base Construction Cost (Accounts 10 to 30)', 'Total Cost (USD)']).values + \
        (db.loc[db['Title'] == '40s - Subtotal', 'Total Cost (USD)']).values+\
        (db.loc[db['Title'] == '50s - Subtotal', 'Total Cost (USD)']).values

    (db.loc[db['Title'] == 'Total Capital Investment Cost (All Accounts)', 'Total Cost (USD)']) =\
       (db.loc[db['Title'] == 'Total Overnight Cost (Accounts 10 to 50)', 'Total Cost (USD)']).values +\
        (db.loc[db['Title'] == '60s - Subtotal', 'Total Cost (USD)']).values

   
    # update final results per kw
    (db.loc[db['Title'] == '(Accounts 10 to 20) US$/kWe', 'Total Cost (USD)']) =\
        (db.loc[db['Title'] == 'Total Direct Capital Cost (Accounts 10 to 20)', 'Total Cost (USD)']).values/reactor_power 

    (db.loc[db['Title'] == '(Accounts 10 to 30) US$/kWe', 'Total Cost (USD)']) =\
        (db.loc[db['Title'] == 'Base Construction Cost (Accounts 10 to 30)', 'Total Cost (USD)']).values/reactor_power 
   
    (db.loc[db['Title'] == '(Accounts 10 to 50) US$/kWe', 'Total Cost (USD)']) =\
        (db.loc[db['Title'] == 'Total Overnight Cost (Accounts 10 to 50)', 'Total Cost (USD)']).values/reactor_power 
    
    (db.loc[db['Title'] == '(Accounts 10 to 60) US$/kWe', 'Total Cost (USD)']) =\
        (db.loc[db['Title'] == 'Total Capital Investment Cost (All Accounts)', 'Total Cost (USD)']).values/reactor_power
    	
    return db

## Section - 2 : User Inputs

### Section - 2 - 1 : User-defined Independent Variables (Global Levers)

In [5]:
## ## User-defined Independent Variables (Global Levers)

# land cost
# From the SA report: the cost $22,000 per acre. The land area is 500 acres including recommended buffer
land_cost_per_acre =  0.8*22000 # dollars/acre


# number of firm orders
num_orders = 20

# #investment tax credits subsidies
ITC  =  0.3#

# interest rate :
interest_rate = 0.06 

# cross_site_standardization :
standardization = 0.7 # 0.7 corresponds to 70% standardization for PWRs


# th of a kind
n_th = 1

# This is a hypothetical well-executed project taking 64 months (TIMCAT simulation)
baseline_construction_duration = 64 # months

# start up duration (months)
startup = 12

if n_th == 1:
    design_completion = 0.9 # 1 means 100%
    # #procurement service experience (supply chain experience)
    proc_exp = 1 # 2 means procurement experts. This is ideal. 
    
    # #  architecture and engineeringexperience
    ae_exp= 1
    
    # #  Construction service experience
    ce_exp = 1
    # modularity (applied on civil construction only)
    mod = "stick_built"
    # # Determining if the turbine and containemnt are non-nuclear or nuclear grade equipment (safety related)
    turbine_grade = "nuclear"
    containment_grade = "nuclear"

    



# Change the levers if n >1
if n_th >1:
    design_completion = 1 
    mod = "modularized"
    # # Determining if the turbine and containemnt are non-nuclear or nuclear grade equipment (safety related)
    turbine_grade = "non_nuclear"
    containment_grade = "non_nuclear"

if n_th ==2:
    proc_exp = 1 
    ae_exp = 1
    ce_exp = 1

if n_th >=2:
    proc_exp = 2 
    ae_exp = 2
    ce_exp = 2 


# labor productivity factor = f(construction experience level)
productivity = 0.25*ce_exp + 0.5

global_levers = pd.DataFrame()
global_levers.loc[:, 'Lever'] = ['Baseline Construction Duration (months)', 'Design Completion',\
                                  'Procurement (supply chain) experience ','Architecture & Engineering Experience',\
                                 'Construction service experience',' Land Cost Per Acre (2023 USD)',\
                                'ITC ', ' Interest Rate', 'Turbine grade ', 'Containment grade', 'modulariziation',\
                                 'standardization' , 'productivity',"#th of a kind", "Startup duration (months)"]

global_levers.loc[:, 'User-Input Value'] = [baseline_construction_duration, design_completion, proc_exp, ae_exp, ce_exp, 
                                            land_cost_per_acre, ITC, interest_rate, turbine_grade  , containment_grade , mod,\
                                            standardization, productivity ,n_th, startup] 

global_levers.loc[:, 'Lever baseline value (for a hopothetical well-executed project)'] = [64, 1, 2, 2, 2, 22000, 0, 0.06 ,\
                                                                                           'nuclear', 'nuclear', "stick_built", 0.7 , 1 , 1, 12 ]

global_levers.loc[:, 'Range'] = ['30 - 130', '0 - 1', '0 - 2', '0 - 2','0 - 2' , '1000 - 100000', '0 - 0.5', '0 - 0.3',\
                                 "nuclear or non-nuclear", "nuclear or non-nuclear",  'stick_built or modularized', '0 : 1','0 : 1',
                                '1 - 1000', '3 : 24']



global_levers_changes = global_levers[global_levers['User-Input Value'] != global_levers['Lever baseline value (for a hopothetical well-executed project)']]

slice_ = pd.IndexSlice[global_levers_changes .index, global_levers_changes .columns]
global_levers_styled = (global_levers.style.set_properties(**{'background-color': 'yellow'}, subset=slice_ )\
                            .set_caption("User-Input Global levers <br> (highlighted in yellow if different from the baseline) <br>")\
                        .set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'blue'),
        ('font-size', '20px')
    ]
}]))


global_levers_styled.hide()

Lever,User-Input Value,Lever baseline value (for a hopothetical well-executed project),Range
Baseline Construction Duration (months),64,64,30 - 130
Design Completion,0.900000,1,0 - 1
Procurement (supply chain) experience,1,2,0 - 2
Architecture & Engineering Experience,1,2,0 - 2
Construction service experience,1,2,0 - 2
Land Cost Per Acre (2023 USD),17600.000000,22000,1000 - 100000
ITC,0.300000,0,0 - 0.5
Interest Rate,0.060000,0.060000,0 - 0.3
Turbine grade,nuclear,nuclear,nuclear or non-nuclear
Containment grade,nuclear,nuclear,nuclear or non-nuclear


### Section - 2 - 2 : User-defined Account-Based Variables

In [6]:
# Creating the table for the account based variables
accounts_vars = pd.DataFrame()
accounts_vars.loc[:, 'Account'] = SFR_data.loc[:, 'Account']
accounts_vars.loc[:, 'Title'] = SFR_data.loc[:, 'Title']

accounts_vars["Design Maturity"] = None
accounts_vars["Supply Chain Delay (monthhs)"] = None

accounts_vars = accounts_vars.loc[accounts_vars['Account'].isin([21, 22, 23, 24, 25, 26])]


# Assigning independent account-based variables
# design_maturity = 2 # 0 if immature (never done). 2 if mature and ready. 1: in between


if n_th <2:
    accounts_vars['Design Maturity'] = [0, 0, 0, 0, 0, 0] 


if n_th >=2:
    accounts_vars['Design Maturity'] = [2, 2, 2, 2, 2, 2] 
    


# Assigning dependent account-based variables

# Supply chain delay(months)= supply chain delay due to design immaturity (months) * delayfactor due to procurement inexperience
# supply chain delay due to design immaturity (months)= -6*design immaturity+12
# delayfactor due to procurement inexperience=-0.25*PE+1.5

accounts_vars["Supply Chain Delay (monthhs)"] = (- 6 *(accounts_vars['Design Maturity'])+12) * (-0.25*proc_exp +1.5) # in months




accounts_vars.loc[len(accounts_vars)] = pd.Series(dtype='float64')
accounts_vars = accounts_vars.fillna('') # remove nan

accounts_vars.loc[len(accounts_vars.index)] = \
    ['', 'Baseline Value = ', 2, 0] 

accounts_vars.loc[len(accounts_vars.index)] = \
    ['', ' ', "Range = 0 : 1", "Range = 0 : 18"]  


accounts_vars

accounts_vars_styled = accounts_vars.style.apply(highlight_changes, axis=1, subset=pd.IndexSlice[:, ['Design Maturity']], baseline = 2)\
    .apply(highlight_changes, axis=1, subset=pd.IndexSlice[:, ['Supply Chain Delay (monthhs)']], baseline =0)\
    .set_caption("User-Input Account-Based levers <br> (highlighted in yellow if different from the baseline)").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'red'),
        ('font-size', '20px')
    ]}])
accounts_vars_styled.hide()

Account,Title,Design Maturity,Supply Chain Delay (monthhs)
21,Structures & Improvements,0.000000,15.000000
22,Reactor System,0.000000,15.000000
23,Energy Conversion System,0.000000,15.000000
24,Electrical Equipment,0.000000,15.000000
25,Initial fuel inventory,0.000000,15.000000
26,Miscellaneous Equipment,0.000000,15.000000
,,,
,Baseline Value =,2,0
,,Range = 0 : 1,Range = 0 : 18


### Section - 2 - 3 : Cost reduction framework (NEED TO BE UPDATED!!!)

<center><table>
    <tr>
        <th><img src="./diagram.png",align="middle",height="10000"/></th>
    </tr>
</table>
</center>

## Section - 3 : Updating the Cost Summary based on user inputs

### Section 3-1 : The land cost & Taxes

In [7]:
# The cost of the land is 22000$ per acre (500 acres
# The cost is multiplied by the new $/acre divided by the old one
# Accounts 11 and 12 are changed


db = pd.DataFrame()
db = SFR_data_0[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()


db.loc[db.Account == 11, 'Total Cost (USD)'] = None # clear old values
db.loc[db.Account == 12, 'Total Cost (USD)'] = None # clear old values
db.loc[db.Account == 51, 'Total Cost (USD)'] = None # clear old values


db.loc[db.Account == 11, 'Total Cost (USD)'] = (land_cost_per_acre /22000)*(SFR_data_0.loc[SFR_data_0.Account == 11, 'Total Cost (USD)'].values)
db.loc[db.Account == 12, 'Total Cost (USD)'] = (land_cost_per_acre /22000)*(SFR_data_0.loc[SFR_data_0.Account == 12, 'Total Cost (USD)'].values)


# The taxes scale with increasing the land cost 
db.loc[db.Account == 51, 'Total Cost (USD)'] = (land_cost_per_acre /22000)*(SFR_data_0.loc[SFR_data_0.Account == 51, 'Total Cost (USD)'].values)


SFR_data_updated_1 = update_high_level_costs(db)


SFR_data_updated_1_ = pd.DataFrame()
SFR_data_updated_1_ = SFR_data_updated_1[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

SFR_data_updated_1_pretty = prettify(SFR_data_updated_1_, "SFR-FOAK Capital Cost Summary - Updated ", 'no_subsidies')

SFR_data_updated_1_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 8,800,000",,,,
12,Site Permits,"$ 1,279,113",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 76,471,787",,,,


### Section 3-2 : Whether the containment and turbine are nuclear grade equipment

In [8]:
# If nuclear (the default), the materials cost is multiplied by 1.5, the labor cost is 1.3
# if non-nuclear, the cost reduction factors for the materials and labor are 0.67 and 0.77 respectively

db = pd.DataFrame()

db = SFR_data_updated_1[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

db.loc[db.Account == 212, 'Site Material Cost'] = None # clear old values
db.loc[db.Account == 212, 'Site Labor Cost'] = None # clear old values
db.loc[db.Account ==  212, 'Site Labor Hours'] = None
db.loc[db.Account ==  213, 'Site Material Cost'] = None
db.loc[db.Account ==  213, 'Site Labor Cost']  = None
db.loc[db.Account ==  213, 'Site Labor Hours'] = None

if containment_grade == "non_nuclear":
    db.loc[db.Account == 212, 'Site Material Cost'] = 0.67*((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 212, 'Site Material Cost']).values)
    db.loc[db.Account == 212, 'Site Labor Cost'] = 0.77*((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 212, 'Site Labor Cost']).values)
    db.loc[db.Account == 212,'Site Labor Hours'] = 0.77*((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 212, 'Site Labor Hours']).values)

else:
    db.loc[db.Account == 212, 'Site Material Cost'] = ((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 212, 'Site Material Cost']).values)
    db.loc[db.Account == 212, 'Site Labor Cost'] = ((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 212, 'Site Labor Cost']).values)
    db.loc[db.Account == 212,'Site Labor Hours'] = ((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 212, 'Site Labor Hours']).values)

if turbine_grade == "non_nuclear":
    db.loc[db.Account == 213, 'Site Material Cost'] = 0.67*((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 213, 'Site Material Cost']).values)
    db.loc[db.Account == 213, 'Site Labor Cost'] = 0.77*((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 213, 'Site Labor Cost']).values)
    db.loc[db.Account == 213, 'Site Labor Hours'] = 0.77*((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 213, 'Site Labor Hours']).values)

else:
    db.loc[db.Account == 213, 'Site Material Cost'] = ((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 213, 'Site Material Cost']).values)
    db.loc[db.Account == 213, 'Site Labor Cost'] = ((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 213, 'Site Labor Cost']).values)
    db.loc[db.Account == 213, 'Site Labor Hours'] = ((SFR_data_updated_1.loc[SFR_data_updated_1.Account == 213, 'Site Labor Hours']).values)


SFR_data_updated_2 = update_high_level_costs(db)

SFR_data_updated_2_ = pd.DataFrame()
SFR_data_updated_2_ = SFR_data_updated_2[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

SFR_data_updated_2_pretty = prettify(SFR_data_updated_2_, "SFR-FOAK Capital Cost Summary - Updated ", 'no_subsidies')
SFR_data_updated_2_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 8,800,000",,,,
12,Site Permits,"$ 1,279,113",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 76,471,787",,,,


### Section 3-3  : Modularity

In [9]:
# If TRUE, increase factory cost by <factor>, reduce site material hours by <factor>, and reduce site labor hours & cost by <factor>.
# This applies only on accuount 21
# we apply the modularity on acoounts 212, 213, 211 plus 214 to 219	 and account 21 updates automatically
# The multipliers are 1.5, 0.5. 0.5

db = pd.DataFrame()

db = SFR_data_updated_2[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()



# remove old stuff

for x in [212, 213, '211 plus 214 to 219']:  
    db.loc[db.Account == x, 'Factory Equipment Cost'] = None # clear old values
    db.loc[db.Account == x, 'Site Labor Hours'] = None
    db.loc[db.Account == x, 'Site Labor Cost'] = None
    db.loc[db.Account == x, 'Site Material Cost'] = None



for x in [212, 213, '211 plus 214 to 219']: 
    if mod == 'modularized':
        db.loc[db.Account == x, 'Factory Equipment Cost'] =\
        ((SFR_data_updated_2.loc[SFR_data_updated_2.Account == x, 'Factory Equipment Cost']).values)*1.5
        
        db.loc[db.Account == x, 'Site Labor Hours'] =\
        ((SFR_data_updated_2.loc[SFR_data_updated_2.Account == x, 'Site Labor Hours']).values)*0.5
        
        db.loc[db.Account == x, 'Site Labor Cost'] =\
        ((SFR_data_updated_2.loc[SFR_data_updated_2.Account == x, 'Site Labor Cost']).values)*0.5
    
        db.loc[db.Account == x, 'Site Material Cost'] =\
        ((SFR_data_updated_2.loc[SFR_data_updated_2.Account == x, 'Site Material Cost']).values)*0.5

    else:
        db.loc[db.Account == x, 'Factory Equipment Cost'] =\
        ((SFR_data_updated_2.loc[SFR_data_updated_2.Account == x, 'Factory Equipment Cost']).values)*1
        
        db.loc[db.Account == x, 'Site Labor Hours'] =\
        ((SFR_data_updated_2.loc[SFR_data_updated_2.Account == x, 'Site Labor Hours']).values)*1
        
        db.loc[db.Account == x, 'Site Labor Cost'] =\
        ((SFR_data_updated_2.loc[SFR_data_updated_2.Account == x, 'Site Labor Cost']).values)*1
    
        db.loc[db.Account == x, 'Site Material Cost'] =\
        ((SFR_data_updated_2.loc[SFR_data_updated_2.Account == x, 'Site Material Cost']).values)*1

SFR_data_updated_3 = update_high_level_costs(db)
SFR_data_updated_3_ = pd.DataFrame()
SFR_data_updated_3_ = SFR_data_updated_3[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

SFR_data_updated_3_pretty = prettify(SFR_data_updated_3_, "SFR-FOAK Capital Cost Summary - Updated ", 'no_subsidies')
SFR_data_updated_3_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 8,800,000",,,,
12,Site Permits,"$ 1,279,113",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 76,471,787",,,,


### Section 3-4 : Bulk Ordering

In [10]:
db = pd.DataFrame()

db = SFR_data_updated_3[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()



# Creating the table for the learning rates
#These rates are from KS-TIMCAT results.
# The learning rates are multiplied by the standardization divded by 0.7 (since the standatization of PWRs was 0.7)
fitted_LR = pd.DataFrame()
fitted_LR.loc[:, 'Account'] = SFR_data.loc[:, 'Account']
fitted_LR.loc[:, 'Title'] = SFR_data.loc[:, 'Title']
fitted_LR = fitted_LR.loc[fitted_LR['Account'].isin([212, 213, '211 plus 214 to 219', 22, '232.1', 233, 24, 26])]

fitted_LR['Factory LR'] = np.array([0.014, 0.014, 0.014, 0.184, 0.27, 0.27, 0.014, 0.014])*standardization/0.7
fitted_LR['Mat LR'] =     np.array([0.1, 0.1, 0.1, 0.081, 0, 0, 0.1, 0.1])*standardization/0.7
fitted_LR['Lab LR'] =     np.array([0.185, 0.185, 0.185, 0.149, 0.139, 0.139, 0.185, 0.185])*standardization/0.7



for x in [212, 213, '211 plus 214 to 219', 22, '232.1', 233, 24, 26]:  
    db.loc[db.Account == x, 'Factory Equipment Cost'] = None # clear old values
    db.loc[db.Account == x, 'Site Labor Hours'] = None
    db.loc[db.Account == x, 'Site Labor Cost'] = None
    db.loc[db.Account == x, 'Site Material Cost'] = None


# # # Bulk order reduction	
for x in [212, 213, '211 plus 214 to 219', 22, '232.1', 233, 24, 26]:
    fac_cost_reduction_multiplier = (1 - (fitted_LR.loc[fitted_LR.Account == x, 'Factory LR'].values[0]))**np.log2(n_th)
    mat_cost_reduction_multiplier = (1 - (fitted_LR.loc[fitted_LR.Account == x, 'Mat LR'].values[0]))**np.log2(n_th)
    lab_cost_reduction_multiplier = (1 - (fitted_LR.loc[fitted_LR.Account == x, 'Lab LR'].values[0]))**np.log2(n_th)
    
    db.loc[db.Account == x, 'Factory Equipment Cost'] =\
        (( SFR_data_updated_3.loc[ SFR_data_updated_3.Account == x, 'Factory Equipment Cost']))* fac_cost_reduction_multiplier
    
    db.loc[db.Account == x, 'Site Material Cost'] =\
        (( SFR_data_updated_3.loc[ SFR_data_updated_3.Account == x, 'Site Material Cost']))* mat_cost_reduction_multiplier

    db.loc[db.Account == x, 'Site Labor Hours'] =\
        (( SFR_data_updated_3.loc[ SFR_data_updated_3.Account == x, 'Site Labor Hours']))* lab_cost_reduction_multiplier
   
    db.loc[db.Account == x, 'Site Labor Cost'] =\
        (( SFR_data_updated_3.loc[ SFR_data_updated_3.Account == x, 'Site Labor Cost']))* lab_cost_reduction_multiplier


SFR_data_updated_4 = update_high_level_costs(db)
SFR_data_updated_4_ = pd.DataFrame()
SFR_data_updated_4_ = SFR_data_updated_4[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

SFR_data_updated_4_pretty = prettify(SFR_data_updated_4_, "SFR-FOAK Capital Cost Summary - Updated ", 'no_subsidies')
SFR_data_updated_4_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 8,800,000",,,,
12,Site Permits,"$ 1,279,113",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 76,471,787",,,,


### Section 3-5 : Reworking and labor productivity

In [11]:
#Reworking = f(AE, CE, design completion)
#(-0.25*AE+1.5) * (-0.42*CE+1.84)*(-8.7*design completion+9. 7)

reworking_factor = (-0.26*ae_exp+1.52) * (-0.43*ce_exp+1.85) *(-1.4*design_completion+2.4)


db = pd.DataFrame()

db = SFR_data_updated_4[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

for x in [212, 213, '211 plus 214 to 219',22, '232.1', 233, 24, 26]:  
    db.loc[db.Account == x, 'Factory Equipment Cost'] = None # clear old values
    db.loc[db.Account == x, 'Site Labor Hours'] = None
    db.loc[db.Account == x, 'Site Labor Cost'] = None
    db.loc[db.Account == x, 'Site Material Cost'] = None

for x in [212, 213, '211 plus 214 to 219',22, '232.1', 233, 24, 26]: 
    db.loc[db.Account == x, 'Factory Equipment Cost'] =\
        ((( SFR_data_updated_4.loc[ SFR_data_updated_4.Account == x, 'Factory Equipment Cost']).values)[0])*reworking_factor  
    db.loc[db.Account == x, 'Site Labor Hours'] =\
        ((( SFR_data_updated_4.loc[ SFR_data_updated_4.Account == x, 'Site Labor Hours']).values)[0])*reworking_factor/productivity
    db.loc[db.Account == x, 'Site Labor Cost'] =\
        ((( SFR_data_updated_4.loc[ SFR_data_updated_4.Account == x, 'Site Labor Cost']).values)[0])*reworking_factor/productivity 
    db.loc[db.Account == x, 'Site Material Cost'] =\
        ((( SFR_data_updated_4.loc[ SFR_data_updated_4.Account == x, 'Site Material Cost']).values)[0])*reworking_factor 


# update the construction duration

SFR_data_updated_5 = update_high_level_costs(db)
SFR_data_updated_5_ = pd.DataFrame()
SFR_data_updated_5_ = SFR_data_updated_5[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

SFR_data_updated_5_pretty = prettify(SFR_data_updated_5_, "SFR-FOAK Capital Cost Summary - Updated ", 'no_subsidies')
SFR_data_updated_5_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 8,800,000",,,,
12,Site Permits,"$ 1,279,113",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 76,471,787",,,,


### Section 3-6 :Update construction duration from labor hours

In [12]:
#sum of labor hours for Account 20 in the initial estimation (well exectued scenario)
sum_old_lab_hrs = (SFR_data_0.loc[SFR_data_0.Account == 21, 'Site Labor Hours']).values +\
(SFR_data_0.loc[SFR_data_0.Account == 22, 'Site Labor Hours']).values +\
(SFR_data_0.loc[SFR_data_0.Account == 23, 'Site Labor Hours']).values+\
(SFR_data_0.loc[SFR_data_0.Account == 24, 'Site Labor Hours']).values+\
(SFR_data_0.loc[SFR_data_0.Account == 26, 'Site Labor Hours']).values



# #sum of labor hours for Account 20 in the new estimation 
sum_new_lab_hrs = (db.loc[db.Account == 21, 'Site Labor Hours']).values +\
(db.loc[db.Account == 22, 'Site Labor Hours']).values +\
(db.loc[db.Account == 23, 'Site Labor Hours']).values+\
(db.loc[db.Account == 24, 'Site Labor Hours']).values+\
(db.loc[db.Account == 26, 'Site Labor Hours']).values



# # # change in labor hours for account 20
labor_hour_ratio = (sum_new_lab_hrs)/sum_old_lab_hrs   # note that this number can be positive or negative
labor_hour_ratio 

# 	From the literature   we know that if labor hours changed from 3.8M hours to 20.5M hours (5.4 times), the construction duration changes from 33.2 months to 74.3 months (2.2 times).
# 	We also know that if the labor hours multiplier =1, the cons duration multiplier should be 1.
# 	Using these two points: 

# construction duration multiplier = 0.3 * the labor hours multiplier+0.7

actual_construction_duration = baseline_construction_duration*(0.3*labor_hour_ratio+0.7)

### Section 3-7 supply chain delays

In [13]:
# For the accounts 21, 22, 23, 24, 25, 26, 
# The delays are D_21,D_22,D_23,D_24,D_25,D_26 
# The tasks lengths (in months) are B_21,B_22,B_23,B_24,B_25,B_26

B_21 =  42.1 # months 
B_22 = 60.2
B_23 = 14.8
B_24 = 3.6
B_25 = 10.1
B_26 = 43.9


# The delays are 
D_21 =  accounts_vars["Supply Chain Delay (monthhs)"].values[0]
D_22 =  accounts_vars["Supply Chain Delay (monthhs)"].values[1]
D_23 =  accounts_vars["Supply Chain Delay (monthhs)"].values[2]
D_24 =  accounts_vars["Supply Chain Delay (monthhs)"].values[3]
D_25 =  accounts_vars["Supply Chain Delay (monthhs)"].values[4]
D_26 =  accounts_vars["Supply Chain Delay (monthhs)"].values[5]


# The tasks completion times (in months) are T_21,T_22,T_23,T_24,T_25,T_26
T_21 = B_21 + D_21 
T_22 = 0.09*(B_21+D_21)  +B_22+D_22
T_23 = 0.24*(B_21+D_21)  +B_23+D_23
T_24 = 0.24*(B_21+D_21) + 0.34*(B_23+D_23)  +B_24+D_24
T_25 = 0.18*(B_21+D_21)  +B_25+D_25
T_26 = 0.21*(B_21+D_21)   +B_26+D_26
T_end = max(T_21, T_22, T_23, T_24, T_25, T_26)
T_end
supply_chain_delay = max( T_end - baseline_construction_duration, 0)
final_construction_duration = actual_construction_duration + supply_chain_delay
print("\nThe actual construction duration is estimated to be : ", int(final_construction_duration ), "months\n") 


The actual construction duration is estimated to be :  113 months



### Section 3-8 Calculate the Indirect Cost

In [14]:
#     I got this from Jia
# account 31 cost = (direct material *direct labor hours *0.785/cons_duration/160/1058 + (direct labor cost*0.36)


db = pd.DataFrame()

db = SFR_data_updated_5[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

for x in [31, 32, 33, 34, 35]:  
    db.loc[db.Account == x, 'Total Cost (USD)'] = None # clear old values


# direct mat cost
# #sum of labor hours for Account 20 in the new estimation 
sum_new_mat_cost = (db.loc[db.Account == 21, 'Site Material Cost']).values +(db.loc[db.Account == 22, 'Site Material Cost']).values+\
(db.loc[db.Account == 23, 'Site Material Cost']).values + (db.loc[db.Account == 24, 'Site Material Cost']).values\
+(db.loc[db.Account == 26, 'Site Material Cost']).values 

    
# direct labor hours: sum_new_lab_hrs 


# direct labor cost
# #sum of labor hours for Account 20 in the new estimation 
sum_new_labor_cost = (db.loc[db.Account == 21, 'Site Labor Cost']).values +\
(db.loc[db.Account == 22,  'Site Labor Cost']).values +\
(db.loc[db.Account == 23,  'Site Labor Cost']).values +\
(db.loc[db.Account == 24,  'Site Labor Cost']).values +\
(db.loc[db.Account == 26,  'Site Labor Cost']).values 
    



db.loc[db.Account == 31, 'Total Cost (USD)'] = (sum_new_mat_cost*sum_new_lab_hrs *0.785)/(final_construction_duration * 160 * 1058) +\
sum_new_labor_cost*0.36

db.loc[db.Account == 32, 'Total Cost (USD)'] = sum_new_labor_cost*0.36*3.661


# Accounts 33, 34, 35 increase with increasing 32.
cost_32_multiplier  = db.loc[db.Account == 32, 'Total Cost (USD)'] / SFR_data_0.loc[SFR_data_0.Account == 32, 'Total Cost (USD)']


for x in [33, 34, 35]:
     db.loc[db.Account == x, 'Total Cost (USD)'] =\
    (SFR_data_updated_5.loc[SFR_data_updated_5.Account == x, 'Total Cost (USD)'])*cost_32_multiplier.values[0]
    


SFR_data_updated_6 = update_high_level_costs(db)
SFR_data_updated_6_ = pd.DataFrame()
SFR_data_updated_6_ = SFR_data_updated_6[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

SFR_data_updated_6_pretty = prettify(SFR_data_updated_6_, "SFR-FOAK Capital Cost Summary - Updated ", 'no_subsidies')
SFR_data_updated_6_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 8,800,000",,,,
12,Site Permits,"$ 1,279,113",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 76,471,787",,,,


### Section 3-7 :  Insurance

In [15]:
# insurance increases linearly when increaing the sum of the 20s and 30s account
db = pd.DataFrame()

db = SFR_data_updated_6[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()
db0 = SFR_data_0
db.loc[db.Account == 52, 'Total Cost (USD)'] = None # clear old values

change_in_insuance_cost = (db.loc[db.Title =='20s - Subtotal', 'Total Cost (USD)'].values\
                        + db.loc[db.Title =='30s - Subtotal', 'Total Cost (USD)'].values)/ (db0.loc[db0.Title =='20s - Subtotal', 'Total Cost (USD)'].values\
                        + db0.loc[db0.Title =='30s - Subtotal', 'Total Cost (USD)'].values)

db.loc[db.Account == 52, 'Total Cost (USD)'] =  (change_in_insuance_cost[0])* (SFR_data_updated_6.loc[db.Account == 52, 'Total Cost (USD)'])

SFR_data_updated_7 = update_high_level_costs(db)

SFR_data_updated_7_ = pd.DataFrame()
SFR_data_updated_7_ = SFR_data_updated_7[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

SFR_data_updated_7_pretty = prettify(SFR_data_updated_7_, "SFR-FOAK Capital Cost Summary - Updated ", 'no_subsidies')
SFR_data_updated_7_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 8,800,000",,,,
12,Site Permits,"$ 1,279,113",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 76,471,787",,,,


### Section 3-6 :  Interest

In [16]:
# The baseline interest is 6%

# # tot overnight cost = accounts 10 - 50
# # I cheated this from Levi : SC-HTGR-costreduction_LML.xlsx

tot_overnight_cost = (SFR_data_updated_7.loc[SFR_data_updated_7.Title == 'Total Overnight Cost (Accounts 10 to 50)' , 'Total Cost (USD)']).values[0]


# Interest rate from this equation (from Levi)
B =(1+ np.exp((np.log(1+ interest_rate)) * final_construction_duration/12))
C  =((np.log(1+ interest_rate)*(final_construction_duration/12)/3.14)**2+1)
Interest_expenses = tot_overnight_cost*((0.5*B/C)-1)
print(Interest_expenses)
# # db = pd.DataFrame()

db = SFR_data_updated_7[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()
db.loc[db.Account == 62, 'Total Cost (USD)'] = None # clear old values

(db.loc[db.Account == 62, 'Total Cost (USD)']) = Interest_expenses


SFR_data_updated_8 = update_high_level_costs(db)

FR_data_updated_8_ = pd.DataFrame()
SFR_data_updated_8_ = SFR_data_updated_8[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

SFR_data_updated_8_pretty = prettify(SFR_data_updated_8_, "SFR-FOAK Capital Cost Summary - Updated ", 'no_subsidies')
SFR_data_updated_8_pretty

[1.7448045e+09]


Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 8,800,000",,,,
12,Site Permits,"$ 1,279,113",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 76,471,787",,,,


### Section 3 - 8 :  ITC Subsidies

In [17]:
db = SFR_data_updated_8
tot_cap_investment = db.loc[db.Title =='Total Capital Investment Cost (All Accounts)', 'Total Cost (USD)'].values

# #from levi's equation, the multipliers change with the ITC as follows

cost_multiplier = min(-0.9387*ITC+ 1.0046, 1)
ITC_reduced_capital_cost  = cost_multiplier*tot_cap_investment


db1 = pd.DataFrame()
db1 = SFR_data_updated_8[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

db1.loc[db1.Title== 'Total Capital Investment Cost - ITC reduced', 'Total Cost (USD)'] = None
db1.loc[db1.Title== 'Total Capital Investment Cost - ITC reduced (US$/kWe)', 'Total Cost (USD)'] = None


db1.loc[db1.Title== 'Total Capital Investment Cost - ITC reduced', 'Total Cost (USD)'] = ITC_reduced_capital_cost
db1.loc[db1.Title== 'Total Capital Investment Cost - ITC reduced (US$/kWe)', 'Total Cost (USD)'] = ITC_reduced_capital_cost/reactor_power


SFR_data_updated_9 = update_high_level_costs(db1)

SFR_data_updated_9_ = pd.DataFrame()
SFR_data_updated_9_ = SFR_data_updated_9[['Account', 'Title', 'Total Cost (USD)', 'Factory Equipment Cost', 'Site Labor Hours', 'Site Labor Cost',\
                 'Site Material Cost']].copy()

SFR_data_updated_9_pretty = prettify(SFR_data_updated_9_, "SFR-FOAK Capital Cost Summary - Updated ", 'subsidies')
SFR_data_updated_9_pretty

Account,Title,Total Cost (USD),Factory Equipment Cost,Site Labor Hours,Site Labor Cost,Site Material Cost
10,Capitalized Pre-Construction Costs,,,,,
11,Land & Land Rights,"$ 8,800,000",,,,
12,Site Permits,"$ 1,279,113",,,,
13,Plant Licensing,"$ 24,382,988",,,,
14,Plant Permits,"$ 12,679,167",,,,
15,Plant Studies,"$ 12,679,167",,,,
16,Plant Reports,"$ 3,972,186",,,,
18,Other Pre-Construction Costs,"$ 12,679,167",,,,
,,,,,,
,10s - Subtotal,"$ 76,471,787",,,,
